In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import clear_output
from scipy.integrate import solve_ivp

# SIR Model

In [2]:
'''
Showcase Samples:
1. (S_0, I_0, R_0) = (9900, 100, 0) + (b, g) = (1.5, 0.75).
2. (S_0, I_0, R_0) = (9900, 100, 0) + (b, g) = (3.5, 1.75).

Standard Settings:
1. 20x noisy observations per unit time with discretization 1.
2. {0.5, 2, 8} interval lengths - we'll generate datasets to 20.00.
3. {0.05, 0.15} alpha noise level multipliers.
4. 10x randomized seeds.
'''
# generate separate datasets for each possible combination, using maximum precision
for seed in range(10):
    for alpha in [0.05, 0.15]:
        
        # two possible combinations of params (beta, gamma)
        for params in [(1.5, 0.75), (3.5, 1.75)]:
    
            # set a seed for reproducibility
            np.random.seed(seed)

            # what are our parameters + population size?
            b, g, N = params[0], params[1], 10000

            # encode ODEs for solve_ivp data-generation processes
            def SIR(t, y):

                # unpack y
                S, I, R = tuple(y)

                # dS/dt = -b*S*I/N; dI/dt = b*I*S/N - g*I; dR/dt = g*I
                dSdt = -b*S*I/N
                dIdt = +b*S*I/N - g*I
                dRdt = +g*I

                # return only the derivatives
                return np.array([dSdt, dIdt, dRdt])
    
            # generate our data
            t_start, t_end = 0.0, 20.0
            t_steps = np.linspace(start=t_start, stop=t_end, num=20001)
            SIR_init = np.array([9900, 100, 0])
            X = solve_ivp(fun=SIR, t_span=(t_start, t_end), y0=SIR_init, 
                          t_eval=t_steps, atol=1e-10, rtol=1e-10).y.T

            # compute appropriate noise levels based on alpha choice
            sigmas = alpha * (X.max(axis=0) - X.min(axis=0))
            
            # create a copy of X + add the noise
            X_noised = X.copy()
            X_noised += np.random.normal(loc=0.0, scale=sigmas, size=X.shape)
            
            # save time, X_true, X_noised as a .csv for our gold-standard dataset
            data = np.hstack([t_steps.reshape(-1, 1), X_noised, X])
            cols = ["t", "S_obs", "I_obs", "R_obs", "S_true", "I_true", "R_true"]
            
            # save this dataset to our gold-standard datasets list
            df = pd.DataFrame(data=data, columns=cols)
            df.to_csv(f"data/SIR_beta={b}_gamma={g}_alpha={alpha}_seed={seed}.csv", index=False)

# Lorenz Model

In [3]:
'''
Showcase Samples:
1. (X_0, Y_0, Z_0) = (5, 5, 5) + (beta, rho, sigma) = (8/3, 28.0, 10.0).
2. (X_0, Y_0, Z_0) = (5, 5, 5) + (beta, rho, sigma) = (8/3, 6.0, 10.0).

Standard Settings:
1. 20x noisy observations per unit time with discretization 1.
2. {0.5, 2, 8} interval lengths - we'll generate datasets to 20.00.
3. {0.05, 0.15} alpha noise level multipliers.
4. 10x randomized seeds.
'''
# generate separate datasets for each possible combination, using maximum precision
for seed in range(10):
    for alpha in [0.05, 0.15]:
        
        # two possible combinations of params (beta, gamma)
        for params in [(8/3, 28.0, 10.0), (8/3, 6.0, 10.0)]:
            
             # set a seed for reproducibility
            np.random.seed(seed)

            # what are our parameters?
            beta, rho, sigma = params

            # encode ODEs for solve_ivp data-generation processes
            def lorenz(t, y):

                # unpack y
                X, Y, Z = tuple(y)

                # dXdt = sigma * (Y-X); dYdt = x(rho - z) - y; dZdt = xy - beta*z
                dXdt = sigma * (Y-X)
                dYdt = (X * (rho - Z)) - Y
                dZdt = (X*Y) - (beta*Z)

                # return only the derivatives
                return np.array([dXdt, dYdt, dZdt])
            
            # generate our data
            t_start, t_end = 0.0, 20.0
            t_steps = np.linspace(start=t_start, stop=t_end, num=20001)
            y_init = np.array([5.0, 5.0, 5.0])
            X = solve_ivp(fun=lorenz, t_span=(t_start, t_end), y0=y_init, 
                          t_eval=t_steps, atol=1e-10, rtol=1e-10).y.T
            
            # compute appropriate noise levels based on alpha choice
            sigmas = alpha * (X.max(axis=0) - X.min(axis=0))
            
            # create a copy of X + add the noise
            X_noised = X.copy()
            X_noised += np.random.normal(loc=0.0, scale=sigmas, size=X.shape)
            
            # save time, X_true, X_noised as a .csv for our gold-standard dataset
            data = np.hstack([t_steps.reshape(-1, 1), X_noised, X])
            cols = ["t", "X_obs", "Y_obs", "Z_obs", "X_true", "Y_true", "Z_true"]
            
            # save this dataset to our gold-standard datasets list
            df = pd.DataFrame(data=data, columns=cols)
            df.to_csv(f"data/LORENZ_rho={rho}_alpha={alpha}_seed={seed}.csv", index=False)